In [1]:
import sys
from pathlib import Path
import os
cwd = os.getcwd()
parent = str(Path(cwd).parents[0])
sys.path.append(parent)

In [2]:
import pandas as pd
import numpy as np

In [3]:
raw_extract = pd.read_csv(parent + '/data/interventions/2022-05-26_034658-form_7-refset_34-extractions.tsv', sep='\t', encoding="ISO-8859-1")
raw_arms = pd.read_csv(parent + '/data/interventions/2022-05-26_034703-sub_arm-refset_34-sub-extraction.tsv', sep='\t', encoding="ISO-8859-1")
rec_extract = pd.read_csv(parent + '/data/interventions/2022-05-26_034706-form_7-refset_34-final.tsv', sep='\t', encoding="ISO-8859-1")
rec_arms = pd.read_csv(parent + '/data/interventions/2022-05-26_034711-sub_arm-refset_34-sub-final.tsv', sep='\t', encoding="ISO-8859-1")

In [4]:
reconciled_ids = rec_arms.db_id.to_list()

In [5]:
remove_rec = raw_arms[~raw_arms.db_id.isin(reconciled_ids)]

In [6]:
cols = ['db_id', 'trialid', 'type', 'placebo_plus_soc', 'category', 'intervention', 'intervention_plus_soc']

In [7]:
final_int = pd.concat([remove_rec[cols], rec_arms[cols]]).reset_index(drop=True)

In [8]:
final_exp = final_int[final_int.type == 'experimental'].reset_index(drop=True)

In [12]:
final_exp

,db_id,trialid,type,placebo_plus_soc,category,intervention,intervention_plus_soc,HCQ,IVE,CP
0,tri00037,ChiCTR2000029790,experimental,NaN,traditional_medicine,Traditional Medicine,undefined_soc,0,0,0
1,tri00030,ChiCTR2000029814,experimental,NaN,traditional_medicine,Traditional Medicine,undefined_soc,0,0,0
2,tri00028,ChiCTR2000029819,experimental,NaN,traditional_medicine,Traditional Medicine,undefined_soc,0,0,0
3,tri00022,ChiCTR2000029853,experimental,NaN,drug,Azvudine,no,0,0,0
4,tri00011,ChiCTR2000029941,experimental,NaN,traditional_medicine,Traditional Medicine,undefined_soc,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2465,tri10000,LBCTR2021034768,experimental,NaN,drug,Povidone-iodine,no,0,0,0
2466,tri10000,LBCTR2021034768,experimental,NaN,drug,Chlorhexidine gluconate,no,0,0,0
2467,tri01014,NCT04351763,experimental,NaN,drug,Amiodarone,no,0,0,0
2468,tri01014,NCT04351763,experimental,NaN,drug,Verapamil,no,0,0,0


In [10]:
final_exp['HCQ'] = np.where(final_exp.intervention.str.contains('Hydroxychloroquine'), 1, 0)
final_exp['IVE'] = np.where(final_exp.intervention.str.contains('Ivermectin'), 1, 0)
final_exp['CP'] = np.where(final_exp.intervention.str.contains('Convalescent Plasma'), 1, 0)

In [ ]:
final_exp[['db_id', 'HCQ', 'IVE', 'CP']].groupby('db_id').sum().to_csv(parent + '/data/interventions/common_ints.csv')

In [ ]:
ints = final_int[final_int.type == 'experimental'].intervention

In [ ]:
ints

In [ ]:
ints_exp = final_int[final_int.type == 'experimental']

In [ ]:
ints_exp[ints_exp.

In [ ]:
final_int[final_int.intervention == 'Dabur chyawanprash']

In [ ]:
int_list = ints.to_list()

In [ ]:
all_ints = []
for i in int_list:
    if ';' in i:
        all_ints = all_ints + i.split(';')
    else:
        all_ints.append(i)

In [ ]:
import collections
c = collections.Counter(x for x in all_ints if x)

In [ ]:
int_counts = pd.DataFrame.from_dict(dict(c), orient='index').reset_index()
int_counts.columns = ['interventions', 'counts']

In [ ]:
int_counts.sort_values(by='counts', ascending=False).head(10)

In [ ]:
int_counts.to_csv(parent + '/data/interventions/unique_ints.csv')